# Load data (from previous notebook)

In [ ]:
sentences = open("sentences.txt").read().split("@@@")

In [ ]:
len(sentences)

# Create index

In [ ]:
import tantivy

In [ ]:
schema_builder = tantivy.SchemaBuilder()
schema_builder.add_integer_field("id", stored=True)
schema_builder.add_text_field("text", stored=True)
schema = schema_builder.build()

In [ ]:
import os
try:
    os.mkdir("tantivy-index")
except:
    pass

In [ ]:
index = tantivy.Index(schema, "tantivy-index")
writer = index.writer()

In [ ]:
from tqdm.auto import tqdm
for i, t in tqdm(enumerate(sentences)):
    writer.add_document(tantivy.Document(id=i, text=t))

In [ ]:
writer.commit()

# Search

In [ ]:
# Reload the index to ensure it points to the last commit.
index.reload()

In [ ]:
def search(query, index, top=20):
    searcher = index.searcher()
    query = index.parse_query(query, ["text"])
    search_results = searcher.search(query, limit=20).hits
    res = []
    for (score, doc_id) in search_results:
        doc = searcher.doc(doc_id)
        res.append({ "id": doc["id"][0], "text": doc["text"][0], "score": score })

    return(pd.DataFrame(res))

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [ ]:
search("The climate crisis is worse in poorer countries", index)